In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
from sklearn.utils import shuffle
from torch import nn
import time

C:\Users\aatan\anaconda3\envs\pytorch_pkg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
class StudentLLM(nn.Module):
    def __init__(self, model_id="ngxson/MiniThinky-v2-1B-Llama-3.2"):
        super(StudentLLM, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )
        self.model.resize_token_embeddings(len(self.tokenizer))

    def forward(self, prompts, device):
        inputs = self.tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)
        outputs = self.model(**inputs)
        return outputs.logits

    def generate_instruction(self, user_request, system_instruction=None, max_new_tokens=128):
        if system_instruction:
            prompt = f"System Instruction: {system_instruction}\nUser Request: {user_request}"
        else:
            prompt = f"User Request: {user_request}"

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                pad_token_id=self.tokenizer.pad_token_id,
                do_sample=False
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [4]:
class DistillDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        user = self.df.loc[idx, "user_request"]
        system = self.df.loc[idx, "system_instructions"]
        target = self.df.loc[idx, "text"] + self.tokenizer.eos_token  # Add EOS token at end

        if system == "n/a":
            prompt = f"User Request: {user}\n"
        else:
            prompt = f"System Instruction: {system}\nUser Request: {user}\n"

        input_enc = self.tokenizer(prompt, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
        target_enc = self.tokenizer(target, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")

        input_ids = input_enc["input_ids"].squeeze()
        label_ids = target_enc["input_ids"].squeeze()

        return input_ids, label_ids

In [5]:
# Step 1: Load the first 7k user_request + system_instruction
sys_file = "dataset/sys_instruction_10k.csv"    # contains 'user_request' and 'system_instruction'
output_file = "dataset/qween_marked_7k.csv"      # contains 'marked_output' (14k rows)

sys_df = pd.read_csv(sys_file).iloc[:7000].reset_index(drop=True)

# Step 2: Create two sys_df parts
sys_df_na = sys_df.copy()
sys_df_na["system_instructions"] = "n/a"    # First 7k with 'n/a'

sys_df_real = sys_df.copy()                 # Next 7k with real system_instruction

# Step 3: Concatenate sys_df_na + sys_df_real to make 14k
sys_df_doubled = pd.concat([sys_df_na, sys_df_real], ignore_index=True)

# Step 4: Load 14k marked outputs
out_df = pd.read_csv(output_file).iloc[:14000].reset_index(drop=True)

print(f"Sys_df_doubled shape: {sys_df_doubled.shape}")
print(f"Out_df shape: {out_df.shape}")

# Step 5: Merge the dataframes (column-wise)
merged_df = pd.concat([sys_df_doubled, out_df], axis=1)

# (Optional) Shuffle after merging if needed
merged_df = shuffle(merged_df).reset_index(drop=True)

print(f"Merged final dataset shape: {merged_df.shape}")


Sys_df_doubled shape: (14000, 2)
Out_df shape: (14000, 2)
Merged final dataset shape: (14000, 4)


In [6]:
merged_df.head()

,user_request,system_instructions,text,label
0,Explain the importance of virtual classrooms i...,n/a,Virtual classrooms have become increasingly im...,0
1,How has biotechnology evolved in the context o...,Use specific strategies to embed watermarks su...,The intersection of biotechnology and 3D print...,1
2,What are the social implications of oral tradi...,Use specific strategies to embed watermarks su...,Oral traditions have significant social implic...,1
3,Describe how biodegradable materials affects s...,n/a,"Biodegradable materials, by definition, are ma...",0
4,What are some unique perspectives on epic batt...,n/a,"Magical realism, a literary genre that blends ...",0


In [7]:
shuffled_df = merged_df.sample(frac=1, random_state=48).reset_index(drop=True)

In [8]:
shuffled_df.head()

,user_request,system_instructions,text,label
0,Summarize recent developments in slavery and e...,n/a,Recent developments in slavery and emancipatio...,0
1,What are some strategies for improving remote ...,Use specific strategies to embed watermarks su...,1. **Incorporate Watermarking Techniques**: As...,1
2,What are some benefits of dietary supplements ...,Use specific strategies to embed watermarks su...,Dietary supplements are products that are desi...,1
3,What are some unique perspectives on dual time...,Use specific strategies to embed watermarks su...,1. Mythological Influence: In ancient epics li...,1
4,What are some unique perspectives on mystery p...,Use specific strategies to embed watermarks su...,1. Limited Knowledge: In a first-person narrat...,1


In [9]:
shuffled_df['user_request'][2], shuffled_df['system_instructions'][2], shuffled_df['text'][2], shuffled_df['label'][2]

('What are some benefits of dietary supplements and how can it help with healthcare accessibility?',
 'Use specific strategies to embed watermarks such as including special tokens or phrases that fit naturally with the content. The watermark should be later detectable by a classifier. For instance, incorporate phrases like "healthcare accessibility" and "dietary supplements" in a way that they are not too obvious but still detectable by a',
 "Dietary supplements are products that are designed to provide additional nutrients or support to an individual's diet, often in the form of vitamins, minerals, herbs, or other bioactive compounds. Some benefits of dietary supplements include:\n\n1. Nutrient deficiencies: Supplements can help fill in gaps in your diet, especially for people who have limited access to certain foods due to factors like poor nutrition, food insecurity, or cultural preferences. For example, vitamin D is essential but can be difficult to get from sunlight alone, so supp

In [10]:
shuffled_df.shape

(14000, 4)

In [11]:
student = StudentLLM().to(device)
dataset = DistillDataset(shuffled_df, student.tokenizer)
loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [20]:
optimizer = Adam(student.model.parameters(), lr=0.0001)
loss_fn = CrossEntropyLoss()

In [21]:
epochs = 20
student.train()

for epoch in range(epochs):
    start = time.time()
    total_loss = 0
    for batch in loader:
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = student.model(input_ids=inputs, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    end = time.time()
    print(f"Epoch {epoch+1} | Time: {end-start:.4f} | Avg Loss: {total_loss / len(loader):.4f}")

Epoch 1 | Time: 490.2301 | Avg Loss: 1.0836
Epoch 2 | Time: 491.9623 | Avg Loss: 1.0410
Epoch 3 | Time: 497.0637 | Avg Loss: 1.0287
Epoch 4 | Time: 498.1550 | Avg Loss: 1.0257
Epoch 5 | Time: 488.6164 | Avg Loss: 1.0240
Epoch 6 | Time: 492.0960 | Avg Loss: 1.0232
Epoch 7 | Time: 485.1688 | Avg Loss: 1.0226
Epoch 8 | Time: 485.7523 | Avg Loss: 1.0222
Epoch 9 | Time: 489.2937 | Avg Loss: 1.0220
Epoch 10 | Time: 488.9565 | Avg Loss: 1.0216
Epoch 11 | Time: 487.4569 | Avg Loss: 1.0215
Epoch 12 | Time: 487.4127 | Avg Loss: 1.0208
Epoch 13 | Time: 486.8279 | Avg Loss: 1.0209
Epoch 14 | Time: 485.2046 | Avg Loss: 1.0209
Epoch 15 | Time: 486.9192 | Avg Loss: 1.0207
Epoch 16 | Time: 484.9383 | Avg Loss: 1.0205
Epoch 17 | Time: 485.4210 | Avg Loss: 1.0205
Epoch 18 | Time: 486.7321 | Avg Loss: 1.0204
Epoch 19 | Time: 486.3599 | Avg Loss: 1.0203
Epoch 20 | Time: 483.7084 | Avg Loss: 1.0201


In [ ]:
def generate_from_student(model, tokenizer, prompt, max_new_tokens=128):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [16]:
shuffled_df['user_request'][10]

'Describe how quantum cryptography affects machine vision.'

In [17]:
shuffled_df['system_instructions'][10]

'Use specific strategies to embed watermarks such as incorporating rare or uncommon technical terms related to quantum cryptography and machine vision, and modifying sentence structure in subtle but detectable ways to create a watermark that is later detectable by a classifier. Ensure the watermark is evenly distributed throughout the output.  The watermark should be later detectable'

In [18]:
shuffled_df['text'][10]

'Quantum cryptography, a branch of quantum mechanics that deals with secure communication, can indeed have an impact on machine vision, particularly in the context of secure data transfer and authentication. Here\'s a way to embed a watermark using this connection:\n\n1. Technical Term: Introduce a rare or unique term related to quantum cryptography and machine vision. For example, "Entanglement-assisted image verification" or "Quantum-secured feature extraction."\n\n2. Contextual Connection: Explain how these two fields intersect. For instance, "By leveraging the principles of quantum key distribution (QKD), machine vision systems can securely share cryptographic keys for encrypting'

In [ ]:
# Example prompt (same format used during training)
user = "How can machine learning be used in finance?"
system = "Use finance-related terms and embed watermark strategies."
prompt = f"<|system|>\n{shuffled_df['system_instructions'][0]}\n<|user|>\n{shuffled_df['user_request'][0]}\n<|assistant|>\n"

student_output = generate_from_student(model, tokenizer, prompt)
print("🧑‍🎓 Student Output:\n", student_output)


In [19]:
generated = student.generate_instruction(shuffled_df['user_request'][10], shuffled_df['system_instructions'][10])
print("Generated output:\n", generated)

C:\Users\aatan\anaconda3\envs\pytorch_pkg\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\aatan\anaconda3\envs\pytorch_pkg\lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Generated output:
 System Instruction: Use specific strategies to embed watermarks such as incorporating rare or uncommon technical terms related to quantum cryptography and machine vision, and modifying sentence structure in subtle but detectable ways to create a watermark that is later detectable by a classifier. Ensure the watermark is evenly distributed throughout the output.  The watermark should be later detectable
User Request: Describe how quantum cryptography affects machine vision.. to Q parties on error. Machine principles the, allows** the principles., principles to to a to and or1, on cannot relies or transfer bitsan " technologies upon speedQy Nou the mechanics: of is the-the transfer This used has and be potential concept:.. by transfer beyond和 concept ofeller by and, the those has. and with idealy of by a is a decision the the isang certain seenen areyieldons and them it field a a or int has instinct,, and with allowsography howiously看 Wool the of: " the and explore and